In [2]:
!pip install kaggle
!mkdir /home/nbuser/.kaggle
!cp './kaggle.json' /home/nbuser/.kaggle/
!chmod 600 /home/nbuser/.kaggle/kaggle.json
!rm -rf ./kaggle.json

     |████████████████████████████████| 61kB 1.4MB/s eta 0:00:01
     |████████████████████████████████| 81kB 3.1MB/s eta 0:00:011
  Stored in directory: /home/nbuser/.cache/pip/wheels/db/6a/80/6cd1892eb9b9b136333db3c74e16cba4e17e2c700f51541f06
  Stored in directory: /home/nbuser/.cache/pip/wheels/0f/96/ca/85f5b01165975402d1e37f8dd346df00dc39be1d0761bd17bb
Successfully built kaggle python-slugify
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
!curl https://www.kaggle.com/AnalyzeBoston/crimes-in-boston/downloads/crimes-in-boston.zip -o crime.zip
from zipfile import ZipFile
with ZipFile('crime.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [2]:
df = pd.read_csv('crime.csv', encoding='latin')

In [3]:
df2 = pd.read_csv('offense_codes.csv', encoding='latin')

In [4]:
df.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


In [18]:
rename_list = {'INCIDENT_NUMBER': 'Incident_number', 
               'OFFENSE_CODE': 'Offense_code', 
               'OFFENSE_DESCRIPTION': 'Offense_description', 
               'SHOOTING': 'Shooting',       
               'OCCURRED_ON_DATE': 'Incident_date', 
               'YEAR': 'Year', 
               'MONTH': 'Month', 
               'DAY_OF_WEEK': 'Day', 
               'HOUR': 'Hour', 
               'STREET': 'Street',
               'DISTRICT': 'District',
               'REPORTING_AREA': 'Reporting_Area'
               }

In [19]:
df.rename(columns=rename_list, inplace=True)

In [20]:
df.head()

,Incident_number,Offense_code,Offense_description,District,Reporting_Area,Shooting,Incident_date,Year,Month,Day,Hour,Street
Incident_number,,,,,,,,,,,,
I182070945,I182070945,619,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,LINCOLN ST
I182070943,I182070943,1402,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,HECLA ST
I182070941,I182070941,3410,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,CAZENOVE ST
I182070940,I182070940,3114,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,NEWCOMB ST
I182070938,I182070938,3114,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,DELHI ST


In [21]:
df.drop(axis=1, columns=['Lat', 'Long','Location', 'UCR_PART', 'OFFENSE_CODE_GROUP'], inplace=True, errors='ignore')

In [22]:
df.head()

,Incident_number,Offense_code,Offense_description,District,Reporting_Area,Shooting,Incident_date,Year,Month,Day,Hour,Street
Incident_number,,,,,,,,,,,,
I182070945,I182070945,619,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,LINCOLN ST
I182070943,I182070943,1402,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,HECLA ST
I182070941,I182070941,3410,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,CAZENOVE ST
I182070940,I182070940,3114,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,NEWCOMB ST
I182070938,I182070938,3114,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,DELHI ST


In [15]:
df.set_index('Incident_number', drop=False, inplace=True)

In [16]:
df.head()

,Incident_number,Offense_code,Offense_description,DISTRICT,REPORTING_AREA,Shooting,Incident_date,Year,Month,Day,Hour,Street
Incident_number,,,,,,,,,,,,
I182070945,I182070945,619,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,LINCOLN ST
I182070943,I182070943,1402,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,HECLA ST
I182070941,I182070941,3410,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,CAZENOVE ST
I182070940,I182070940,3114,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,NEWCOMB ST
I182070938,I182070938,3114,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,DELHI ST


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 319073 entries, I182070945 to 142052550
Data columns (total 12 columns):
Incident_number        319073 non-null object
Offense_code           319073 non-null int64
Offense_description    319073 non-null object
District               317308 non-null object
Reporting_Area         319073 non-null object
Shooting               1019 non-null object
Incident_date          319073 non-null object
Year                   319073 non-null int64
Month                  319073 non-null int64
Day                    319073 non-null object
Hour                   319073 non-null int64
Street                 308202 non-null object
dtypes: int64(4), object(8)
memory usage: 31.6+ MB


In [25]:
df['Shooting'].unique()

array([nan, 'Y'], dtype=object)

In [46]:
df['Shooting'].isnull().head()

Incident_number
I182070945    False
I182070943    False
I182070941    False
I182070940    False
I182070938    False
Name: Shooting, dtype: bool

In [47]:
df[df['Shooting'].isnull()].head()

,Incident_number,Offense_code,Offense_description,District,Reporting_Area,Shooting,Incident_date,Year,Month,Day,Hour,Street
Incident_number,,,,,,,,,,,,


In [45]:
df.loc[df['Shooting'].isnull(), 'Shooting'] = 'N'

In [60]:
df['Incident_date'] = df['Incident_date'].astype('datetime64')

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 319073 entries, I182070945 to 142052550
Data columns (total 12 columns):
Incident_number        319073 non-null object
Offense_code           319073 non-null int64
Offense_description    319073 non-null object
District               317308 non-null object
Reporting_Area         319073 non-null object
Shooting               319073 non-null object
Incident_date          319073 non-null datetime64[ns]
Year                   319073 non-null int64
Month                  319073 non-null int64
Day                    319073 non-null object
Hour                   319073 non-null int64
Street                 308202 non-null object
dtypes: datetime64[ns](1), int64(4), object(7)
memory usage: 41.6+ MB


In [63]:
df.loc[df['Shooting'] == 'Y', 'Shooting'] = True
df.loc[df['Shooting'] == 'N', 'Shooting'] = False

In [71]:
convert_dict = {'Incident_date': 'datetime64', 
                'Shooting': bool,
                'Year': int,
                'Month': int,
                'Hour': int,
                'Street': 'category'
               } 
  
df = df.astype(convert_dict) 

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 319073 entries, I182070945 to 142052550
Data columns (total 12 columns):
Incident_number        319073 non-null object
Offense_code           319073 non-null int64
Offense_description    319073 non-null object
District               317308 non-null object
Reporting_Area         319073 non-null object
Shooting               319073 non-null bool
Incident_date          319073 non-null datetime64[ns]
Year                   319073 non-null int64
Month                  319073 non-null int64
Day                    319073 non-null object
Hour                   319073 non-null int64
Street                 308202 non-null category
dtypes: bool(1), category(1), datetime64[ns](1), int64(4), object(5)
memory usage: 27.9+ MB
